In [124]:
from decimal import *
import math
from curve_shift import CNT_SEG, alpha, RONE

In [125]:
getcontext().prec = 25

swapFee = Decimal(0)
swapFee = Decimal(0.0035)
afterSwapFee = Decimal(1) - swapFee
protocolFee = Decimal(0.2)
totalSupply = Decimal(0)
MINIMUM_LIQUIDITY = 10**3

In [126]:
def getWeight(T):
    res = alpha[T//3600]
    return (Decimal(RONE - (res * RONE)), Decimal((res * RONE)))

def getRawWeight(T):
    res = alpha[T//3600]
    return (Decimal(1-res), Decimal(res))


In [127]:
################ K STUFF #################################
kLast = -1
totalTreasury = 0

def calcK(T):
    Win, Wout = getRawWeight(T)
    global Bin, Bout
    return round(pow(Bin, Win) * pow(Bout, Wout))

def updateK(T):
    global kLast
    kNow = calcK(T)
    kLast = kNow

def updateTreasury(T):
    global kLast, totalSupply, totalTreasury
    kNow = calcK(T)
    if kNow > kLast:
        numer = totalSupply * (kNow - kLast)
        denom = (1 - protocolFee) * kNow / protocolFee + kLast

        gainedLp = numer / denom
        totalTreasury += gainedLp
        totalSupply += gainedLp
        updateK(T)
        return gainedLp
    updateK(T)

#######################################################

def calRightProtocolLP(Bin, Bout, Win, Wout, constsSwapAmount):
    actual_swap_fee = constsSwapAmount * swapFee
    actual_protocol_fee = actual_swap_fee * protocolFee
    tempBin = Bin + constsSwapAmount - actual_protocol_fee
    tempBout = Bout - calOutWeight(Bin, Bout, Win, Wout, constsSwapAmount)
    t = (tempBin + actual_protocol_fee) / tempBin
    outLp = (pow(tempBin, Win) * pow(tempBout, Wout)) * (pow(t, Win) - 1)
    return outLp

def updateBalance(da, db):    
    global Bin, Bout
    Bin = Bin + da
    Bout = Bout + db

def calOutWeight(Bin, Bout, Win, Wout, inAmount):
    oldInAmount = inAmount
    inAmount = inAmount * afterSwapFee
    res = Bout * (1 - pow((Bin / (Bin + inAmount)), Win / Wout))
    return res



### SWAPPING FUNCTIONS
def swapTokenToXyt(T, inAmount):
    print("\n==================SWAPPING===================")
    global Bin, Bout
    
    Win, Wout = getRawWeight(T)

    alter = calRightProtocolLP(Bin, Bout, Win, Wout, inAmount)
    print("LP alter:", alter)

    updateK(T)
    outAmount = calOutWeight(Bin, Bout, Win, Wout, inAmount)
    updateBalance(inAmount, -outAmount)
    actualLp = updateTreasury(T)

    if actualLp is None:
        return

    print("Actual LP:", actualLp)
    print('')
    print("Difference:", abs(actualLp - alter))
    print("Difference / totalSupply", abs(actualLp - alter)/totalSupply)

    print("============================================\n")


def swapXytToToken(T, inAmount):
    print("\n==================SWAPPING===================")
    global Bin, Bout

    Win, Wout = getRawWeight(T)

    alter = calRightProtocolLP(Bout, Bin, Wout, Win, inAmount)
    print("LP alter:", alter)

    updateK(T)
    outAmount = calOutWeight(Bout, Bin, Wout, Win, inAmount)
    updateBalance(-outAmount, inAmount)
    actualLp = updateTreasury(T)

    if actualLp is None:
        return

    print("Actual LP:", actualLp)
    print('')
    print("Difference:", abs(actualLp - alter))
    print("Difference / totalSupply", abs(actualLp - alter)/totalSupply)

    print("============================================\n")

def addLiquidityDual(T, amount):
    x = amount
    y = amount
    global Bin, Bout, totalSupply
    if Bin < Bout:
        x = amount * Bin / Bout
    else:
        y = amount * Bout / Bin
    Bin += x
    Bout += y
    totalSupply += totalSupply * x / Bin
    updateK(T)

In [128]:
def run_swap(time, swapTime):
    for i in range(swapTime):
        global Bin, Bout
        amount = Decimal(40000000 + 5000000 * i)
        if i % 3 == 0:
            calOutWeight(time, Bin, Bout, amount, False)
        else:
            calOutWeight(time, Bin, Bout, amount, True)
        addLiquidityDual(amount)

In [129]:
Bin = Decimal(10000000000)
Bout = Decimal(10000000000)

constsSwapAmount = Decimal(1500500 * 15)


totalSupply = Decimal.sqrt(Bin * Bout)

T = 3600 
swapTokenToXyt(T, constsSwapAmount)
swapXytToToken(T, constsSwapAmount)
addLiquidityDual(T, constsSwapAmount)


T = 3600 * 10
swapTokenToXyt(T, constsSwapAmount * 2)
swapXytToToken(T, constsSwapAmount * 3)
addLiquidityDual(T, constsSwapAmount * 4)


T = 3600 * 100
swapTokenToXyt(T, constsSwapAmount * 5)
swapXytToToken(T, constsSwapAmount * 6)
addLiquidityDual(T, constsSwapAmount * 7)


T = 3600 * 300
swapTokenToXyt(T, constsSwapAmount * 8)
swapXytToToken(T, constsSwapAmount * 9)
addLiquidityDual(T, constsSwapAmount * 10)


T = 3600 * 500
swapTokenToXyt(T, constsSwapAmount * 11)
swapXytToToken(T, constsSwapAmount * 12)
addLiquidityDual(T, constsSwapAmount * 13)






==================SWAPPING===================
LP alter: 7860.453969033929829101081
Actual LP: 7860.375285722442079266708

Difference: 0.078683311487749834373
Difference / totalSupply 7.868324963976274750096557E-12


==================SWAPPING===================
LP alter: 7877.103845965479625579322
Actual LP: 7877.150413042573719047294

Difference: 0.046567077094093467972
Difference / totalSupply 4.656700380915155187090712E-12


==================SWAPPING===================
LP alter: 15694.71939115287854690940
Actual LP: 15694.52378798731255129385

Difference: 0.19560316556599561555
Difference / totalSupply 1.951642774140574654651366E-11


==================SWAPPING===================
LP alter: 23564.99980433076164222787
Actual LP: 23564.56365803228995424875

Difference: 0.43614629847168797912
Difference / totalSupply 4.351666641502810600154130E-11


==================SWAPPING===================
LP alter: 39286.85758406285505972964
Actual LP: 39282.93020640526533761978

Difference: 3.9

In [130]:
# Bin = Decimal(1000000000)
# Bout = Decimal(1000000000)

# T = 3600
# constsSwapAmount = Decimal(100000000)
# totalSupply = Decimal.sqrt(Bin * Bout)

# # print(getWeight(T))

# print('')

# print(calRightProtocolLP(constsSwapAmount))
# swapTokenToXyt(T, constsSwapAmount)


# print('')




# print(totalTreasury, totalSupply)
# print('')


# swapXytToToken(T, constsSwapAmount)
# swapTokenToXyt(T, constsSwapAmount)
# addLiquidityDual(T, constsSwapAmount)
# print(totalTreasury, totalSupply)
# print('')

# swapXytToToken(T, constsSwapAmount)
# swapTokenToXyt(T, constsSwapAmount)
# addLiquidityDual(T, constsSwapAmount)
# # print(totalTreasury, totalSupply)

# swapXytToToken(T, constsSwapAmount)
# swapTokenToXyt(T, constsSwapAmount)
# addLiquidityDual(T, constsSwapAmount)
# # print(totalTreasury, totalSupply)

# print('')

# print("1 hour precision:")
# print(66497/66501)
# print(133298/132764)
# print(198826/200350)


# firstK = getK(0)
# exit(0)

# addLiquidityDual(Decimal(1000000000))


# print(calOutWeight(3600, Bin, Bout, 
#     Decimal(220000), 
#     False
# ))
# updateK(3600 + 3600)
# exit(0)

# print(calOutWeight(36000, Bin, Bout, 
#     Decimal(25000),
#     True
# ))

# run_swap(36000+3600, 10)
# updateK(36000+3600)

# print(calOutWeight(360000, Bin, Bout, 
#     Decimal(725000),
#     True
# ))
# updateK(360000+3600)

# print(calOutWeight(1692000, Bin, Bout,
#     Decimal(400000),
#     False
# ))
# updateK(1692000+3600)

# run_swap(1692000+3600+3600, 10)
# updateK(1692000+3600+3600+3600)


# lastK = getK(1692000)

# print(firstK, lastK)

x = Decimal(0.6)

print('')
print('')